<a href="https://colab.research.google.com/github/OneFineStarstuff/State-of-the-Art/blob/main/Attention_Mechanisms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

# Define the Attention mechanism
class Attention(nn.Module):
    def __init__(self, input_dim):
        super(Attention, self).__init__()
        self.attn_weights = nn.Linear(input_dim, 1)

    def forward(self, x):
        attn_scores = self.attn_weights(x).squeeze(-1)
        attn_weights = torch.softmax(attn_scores, dim=1)
        attn_output = torch.sum(x * attn_weights.unsqueeze(-1), dim=1)
        return attn_output

# Define the Attention-based classifier
class AttentionClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super(AttentionClassifier, self).__init__()
        self.attn = Attention(input_dim)
        self.fc = nn.Linear(input_dim, hidden_dim)
        self.out = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        x = self.attn(x)
        x = torch.relu(self.fc(x))
        x = self.out(x)
        return x

# Example usage
input_dim = 128
hidden_dim = 64
num_classes = 10

# Initialize the model
model = AttentionClassifier(input_dim, hidden_dim, num_classes)

# Generate random input data (batch size, sequence length, input_dim)
input_data = torch.randn(32, 10, input_dim)

# Get the model output
output = model(input_data)
print(output.shape)  # Expected output shape: (batch size, num_classes)